# Generating reports with mingo

The following demo presents a simple program that creates a new database, fills it with the datafiles in `./sources` and generates a report with the data it contains.

Though this program only has a couple of lines of code, it does a lot of things under the hood, so it may take around 7 minutes to execute.

## Import required functions

You will only need to import `Database` and `report` for this demo.

- `Database` is a class used to create, fill and delete databases.
- `report` is a function used to generate a PDF report from the data contained in the database.

In [ ]:
from mingo import Database, report

## Create empty database

The first step is to create a new database, which we will call `demo`. This action is performed by the constructor of `Database` --i.e. the function that is called when you initialize an object of `Database` class.

Aside from the name of the database, the constructor needs some information about the mariadb user and server in order to create it. The easiest way to provide this information is to create a MariaDB [option file](https://mariadb.com/kb/en/configuring-mariadb-with-option-files/#example-minimal-option-file) and to pass its path to the constructor via the `cnf_path` argument, which is set to `~/.my.cnf` by default.

In [ ]:
db = Database("demo", cnf_path="~/.my.cnf")

If you prefer to avoid option files, you can still pass the required information to the constructor via keyword arguments. You can get the constructor's argument list by hovering over the word `Database` in the previous cell or by executing `print(Database.__doc__)`.

## Fill the database using data files

The `fill` method of the `Database` class can be used to read data files and insert their data in the database.

Fill may behave differently depending on the type of its argument:

- Path to file: Read data from file and add it to database.
- Path to directory: Look for data files inside it and repeat previous process.
- Sequence of paths: Perform one of the mentioned actions for each path.

If a directory contains other directories, `fill` will also look inside them.

Files that are not data files will be skipped by the function.


In [ ]:
db.fill("./sources")

## Generate a PDF report

Now that our database contains data, we will use the `report` function to generate a PDF report with the following information:

- Number of hits per event as function of initial energy
- Average depth of the electromagnetic shower as function of initial energy
- Number of hits per plane as a function of initial energy
- Average distance between the hits and the center of each active plane as a function of initial energy

The function takes the `Database` object associated with our database and the path of the PDF that will be generated.

In [ ]:
report(db, "demo-report.pdf")

You should now find a file called `demo-report.pdf` inside `doc/demo`. As you can see, each page of the report contains a plot and a table with statistical properties of the specific data that is being plotted.

You can now use the `drop` method of `Database` to delete the `demo` database from your computer if you don't want to keep it.


In [ ]:
db.drop()

And that's it! You now know how you to create a MariaDB database and a PDF report from a set of source files using `mingo`.